# Data == Music
-------------
A predictive model that can predict how highly rated 
music will be based on audio characteristics and 
track information from Spotify!

Let's recall the steps to making a machine learning model:
1. Create a feature matrix and response vector (X and y)
2. Import an estimator class
3. Instantiate that estimator class
4. Fit the model with training data
5. Use the model to predict a new observation
6. Evaluate the model's accuracy
7. ???
8. PROFIT!

We'll be following steps 1-6 in this notebook.

In [ ]:
# Import and set up ALL THE THINGS!

import numpy as np
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(depth=4)

import matplotlib as mpl
import matplotlib.pyplot as plt 
%matplotlib inline 
plt.rcParams['figure.figsize'] = (12.0, 8.0)

import seaborn as sns
sns.color_palette("muted")
sns.set_style("ticks", 
                {'axes.grid': False,
                 'axes.linewidth': 0.5,
                })

from sklearn import metrics
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor

from DataEqualsMusic import SpotiAPI

First things first, let's get our data from a .csv into a DataFrame.
(FYI, you can update the .csv by running DataEqualsMusic.py.)

In [ ]:
file = "data/modified_spotify_top_200.csv"
df = pd.read_csv(file)

# Here's our DataFrame's top 3 entries
df.head(3)

test = df.drop("Unnamed: 0", axis=1)

test.head(5)

In [ ]:
# Our feature columns are everything after "Position", which is our response variable.
feature_cols = df.columns[2:]

X = df[feature_cols]
y = df.Position

Before we really dig in, let's try to visualize any potential relationships in the data.

Are there any overall trends we can see right off the bat?

In [ ]:
# Let's see if a scatter plot shows us anything.
plt.scatter(df["energy"], y)

# Set axis limits: energy 0.0 - 1.0, Position 1 - 200
plt.axis((0.0,1.0,0,200))

# Invert the y axis, we want high ranking to be on top.
plt.gca().invert_yaxis()

# Vanity
sns.despine()

plt.show()

At least we can see that most of the top 200 tracks tend to be fairly energetic. Let's see where most of them are relative to the others.

In [ ]:
e = df.energy
p_25 = np.percentile(e, 25) # Returns 25th percentile of the "energy" feature
p_50 = np.percentile(e, 50) 
p_75 = np.percentile(e, 75)

# Same plot as before, plus lines for the percentiles
plt.scatter(df["energy"], y)
plt.axis((0.0,1.0,0,200))
plt.gca().invert_yaxis()
sns.despine()

# Just so we can see them, let's give the lines some colors
plt.axvline(x=p_25, color="Green")
plt.axvline(x=p_50, color="Black")
plt.axvline(x=p_75, color="Green")

plt.show()

In [ ]:
print p_25, p_50, p_75

Well well! From this we can see that most tracks are pretty energetic, like we thought. 

The median energy level is 0.7, which about the norm for artists like Selena Gomez and David Guetta. Before we assume anything though, we should notice that some artists (like Rihanna and Drake) rank highly despite having low-energy songs.

In [ ]:
# Let's try the same for danceability
d = df.danceability
p_25 = np.percentile(d, 25)
p_50 = np.percentile(d, 50) 
p_75 = np.percentile(d, 75)
plt.scatter(df["danceability"], y)
plt.axis((0.0,1.0,0,200))
plt.gca().invert_yaxis()
sns.despine()
plt.axvline(x=p_25, color="Green")
plt.axvline(x=p_50, color="Black")
plt.axvline(x=p_75, color="Green")

plt.show()

In [ ]:
print p_25, p_50, p_75

To be honest I'm disappointed that most songs have such low danceability, I expected higher.

In [ ]:
# Find out most predictive feature with correlation matrix
sns.heatmap(df.corr())

A track's Position in the global Top 200 charts seems to be:
* Slightly negatively correlated with the track's key
* Slightly positively correlated with the track's mode and duration

## 1. Create a feature matrix and response vector (X and y)

In [ ]:
# Our feature columns are everything after "Position", which is our response variable.
feature_cols = df.columns[2:]

X = df[feature_cols]
y = df.Position

In [ ]:
# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# Let's take a look at the shape of our data.
print X.shape # 200,14. Makes sense; 200 rows and 14 features.
print y.shape # 200,

print X_train.shape # 150, 14
print X_test.shape # 50, 14
print y_train.shape # 150,
print y_test.shape # 50,

At this point, I attempted to make a logistic regression model, fit it to the data, and make predictions with it. 

However, that turned out to be _incredibly_ non-predictive so I scrapped it. 

Unfortunately, I didn't keep a copy for posterity so we're just going to move on to a model that should perform better: RandomForestRegressor!

## 2. Import an estimator class

In [ ]:
# Technically, we already did this up top but here it is anyway:
from sklearn.ensemble import RandomForestRegressor

## 3. Instantiate that estimator class

In [ ]:
# We'll stick with the default n_estimators=10 for now.
# We'll use the same random state throughout this notebook, 
    # just so that we can be sure no error is due to it.
rf = RandomForestRegressor(random_state=1)

rf

## 4. Fit the model with training data

In [ ]:
# This kind of regressor doesn't need a train/test split, so we'll fit it to all of our data.
rf.fit(X, y)

While we're at it, let's see the feature importances:

In [ ]:
importance = pd.DataFrame({'feature':feature_cols, 'importance':rf.feature_importances_}).sort_values('importance', ascending=False)
importance.sort_values("importance",ascending=False)

## 5. Use the model to predict a new observation

In [ ]:
y_train_prediction = rf.predict(X_train)
y_test_prediction = rf.predict(X_test)

In [ ]:
##

## 6. Evaluate the model's accuracy

In [ ]:
# Untuned, this is the model's accuracy
MSE_scores = cross_val_score(rf, X, y, cv=10, scoring='mean_squared_error')
RMSE_scores.append(np.mean(np.sqrt(-MSE_scores)))

# Let's look at the residuals

np.mean(RMSE_scores)
# y_train_prediction

### Tuning n-estimators

In [ ]:
# Let's tune the model a bit and see where that gets us.

# list of values to try for n_estimators
estimator_range = range(1, 20)

# list to store the average RMSE for each value of max_depth
RMSE_scores = []

# use 5-fold cross-validation with each value of n_estimators (WARNING: SLOW!)
for estimator in estimator_range:
    rfreg = RandomForestRegressor(n_estimators=estimator, random_state=1)
    MSE_scores = cross_val_score(rfreg, X, y, cv=5, scoring='mean_squared_error')
    RMSE_scores.append(np.mean(np.sqrt(-MSE_scores)))
    
# plot n_estimators (x-axis) versus RMSE (y-axis)
plt.plot(estimator_range, RMSE_scores)
plt.xlabel('n_estimators')
plt.ylabel('RMSE (lower is better)')

In [ ]:
# show the best RMSE and the corresponding n_estimators
final_n_estimators = sorted(zip(RMSE_scores, estimator_range))[0][1]
final_n_estimators

### Tuning max features

In [ ]:
# list of values to try for max_features
feature_range = range(1, len(feature_cols)+1)

# list to store the average RMSE for each value of max_features
RMSE_scores = []

# use 10-fold cross-validation with each value of max_features (WARNING: SLOW!)
for feature in feature_range:
    rfreg = RandomForestRegressor(n_estimators=150, max_features=feature, random_state=1)
    MSE_scores = cross_val_score(rfreg, X, y, cv=10, scoring='mean_squared_error')
    RMSE_scores.append(np.mean(np.sqrt(-MSE_scores)))

In [ ]:
# plot max_depth (x-axis) versus RMSE (y-axis)
plt.plot(max_depth_range, RMSE_scores)
plt.xlabel('max_depth')
plt.ylabel('RMSE (lower is better)')

In [ ]:
# Let's visualize the results of our max_features tuning